# Short notebook as a check for comparing two models: Classic SIR and SIR with hospitalizations. 
It should be possible to get an accurate estimate of hospitalizations if they are linear in number of infections and hospitalizations:

Given
$$\dot{H} = r_{chr} \gamma \cdot I(t) - u \cdot H(t)$$
or 
$$\dot{H} + u \cdot H(t) = r_{chr}  \gamma \cdot I(t) $$

with $r_{chr}$ is the case-hospitalization rate and $u$ is the rate at which people leave the hospital.

The analytical solution can be determined:

With integrating factor 
$\mu(t) = e^{u t}$

So:
$e^{ut}H(t) + k = \int e^{ut} r_{chr}  \gamma I(t) dt $

Or, in other words:
$$ H(t) = e^{-ut} \left( r_{chr} \gamma \int e^{ut} I(t) dt - k \right) $$

By calculating $\int e^{ut} I(t) dt $ numerically using the trapezoidal rule, $H(t)$ can be estimated.

<!-- int_I = integrate.cumtrapz(np.exp(u_H*ts)*Is,ts, initial=0)
Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k)  -->

In [1]:
%matplotlib widget

import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf
import PandemiXModelFunctions as pmf


import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
             
             
# from scipy import integrate
# int_I = integrate.cumtrapz(np.exp(u_H*ts)*Is,ts, initial=0)
# Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k) 

In [2]:
import importlib 
importlib.reload(pmf)

<module 'PandemiXModelFunctions' from 'c:\\Users\\rakrpe\\OneDrive - Roskilde Universitet\\Documents\\PandemiX\\GithubRepos\\PandemiX\\modelling\\PandemiXModelFunctions.py'>

In [3]:
# Setup model

# Parameters
pars = {
    'beta' : 1.5/7,
    'gamma': 1/7
}
r_chr = 0.8
u_H = 0.25
pars_full = {
    'beta' : pars['beta'],
    'gamma': pars['gamma'],
    'r_chr': r_chr,
    'u_H': u_H}

# Initial conditions
I_0 = 0.001
R_0 = 0
H_0 = 0
S_0 = 1 - R_0 - I_0 - H_0 
init = {
    'S': S_0,
    'I': I_0
}
init_full = {
    'S': S_0,
    'I': I_0,
    'H': H_0
}

tEnd = 200
curScheme = pmf.Scheme('SIR',init,pars,0,tEnd)
curScheme_full = pmf.Scheme('SIHR',init_full,pars_full,0,tEnd).copy()


d1 = 75
betaMultiplier = 2
curScheme.addChange(pmf.Change(d1,MultiplyParameters={'beta':betaMultiplier,'gamma':1}))
curScheme_full.addChange(pmf.Change(d1,MultiplyParameters={'beta':betaMultiplier,'gamma':1,'r_chr':1,'u_H':1}))

In [4]:
curScheme.simulate()
# curScheme.plot()
# print(curScheme.Changes[0])
# print(curScheme.Changes[1])

In [5]:
curScheme_full.simulate()
# curScheme_full.plot()

# Make comparison

In [6]:
ts = curScheme.result.t
Is = curScheme.result.y[1,:]

k = 0
# Hs = (r_chr *Is/u_H) + k * np.exp(-u_H*ts)

ts2 = curScheme_full.result.t
Is2 = curScheme_full.result.y[1,:]
Hs2 = curScheme_full.result.y[2,:]

# # Numerical integration of I(t) using trapz
# int_I =  np.array([np.trapz(np.exp(u_H*ts[:x])*Is[:x],ts[:x]) for x in np.arange(len(ts))])
# Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k) 


from scipy import integrate
int_I = integrate.cumtrapz(np.exp(u_H*ts)*Is,ts, initial=0)
Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k) 

In [10]:
# fig,(ax1,ax2) = plt.subplots(2,1)
fig,ax1 = plt.subplots()

ax1.plot(ts2,Hs2,label='Calculated during simulation')
ax1.plot(ts,Hs,'--',label='Estimated from results after simulation')
# ax1.plot(ts,Hs*pars['gamma'],'--')

ax1.legend()

# ax2.plot(ts2,Hs2/np.max(Hs2))
# ax2.plot(ts,Hs/np.max(Hs),'--')


# ax2.plot(ts,Hs2/Hs)

# ax1.plot(ts2,Is2)
# ax1.plot(ts,Is,'--')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Old below


$$\dot{H} = r_{chr} \cdot I(t) - u \cdot H(t)$$
or 
$$\dot{H} + u \cdot H(t) = r_{chr} \cdot I(t) $$

Has solution:

$\mu(t) = e^{u t}$

So:
$e^{ut}H(t) + k = \int e^{ut} r_{chr} I(t) dt $

Or, in other words:
$$ H(t) = e^{-ut} \left( r_{chr} \int e^{ut} I(t) dt +k \right)



$$ \int e^{ut} I(t) dt  $$

a = e^{ut}
b = I(t)
int(a) = e^{ut}/u
b' = I'(t) but done numerically using np.gradient


integration by parts:
b * int(a) - int(b' * int(a))
I(t) * e^{ut}/u - int(I'(t) * e^{ut}/u)



int(b) = \int I(t) dt solved by numerical integration (trapezoidal rule) 
a' = -u e^{-ut}

integration by parts:
a * int(b) - int(-ue^{-ut} ) 

In [8]:
# asdf = [np.trapz(np.exp(-u_H*ts[:x])*Is[:x],ts[:x]) for x in np.arange(len(ts))]
# asdf = [np.trapz(np.exp(u_H*ts[:x])*Is[:x],ts[:x]) for x in np.arange(len(ts))]
# # asdf = [np.trapz(Is[:x],ts[:x]) for x in np.arange(len(ts))]
# asdf = np.array(asdf)
# # Hs = (r_chr *Is/u_H) + k * np.exp(-u_H*ts)
# k = 0
# Hs = u_H * (r_chr * asdf + k*np.exp(-u_H*ts))
# Hs = np.exp(-u_H*ts) * (r_chr * asdf + k)
# plt.figure()
# plt.plot(ts,Hs)
# # plt.plot(ts,asdf)


Given
$$\dot{H} = r_{chr} \cdot I - u \cdot H(t)$$
or 
$$\dot{H} + u \cdot H(t) = r_{chr} \cdot I $$

with $r_{chr}$ is the case-hospitalization rate and $u$ is the rate at which people leave the hospital. (Note that $I$ is actually time-dependent, but here assumed piece-wise constant)

The analytical solution can be determined:

With integrating factor 
$\mu(t) = e^{u t}$
then 

$e^{ut}H(t) + k = \frac{1}{u} e^{ut} r_{chr} I$

Or, put differently:
$$ H(t) = \frac{r_{chr} I}{u} + k e^{-ut}$$


$\mu(t) = e^{u t}$

So:
$e^{ut}H(t) + k = \int e^{ut} r_{chr} I(t) dt $

Or, in other words:
$$ H(t) = e^{-ut} \left( r_{chr} \int e^{ut} I(t) dt +k \right)